In [1]:
import pandas as pd
import os
import joblib  # Para guardar los modelos entrenados
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier


# Definimos los algoritmos que vamos a usar
modelos_config = {
    "knn": KNeighborsClassifier(n_neighbors=3), # Puedes ajustar n_neighbors
    "svm": SVC(probability=True, random_state=42),
    "nb": GaussianNB(),
    "rf": RandomForestClassifier(n_estimators=100, random_state=42)
}

# Mapeamos los nombres de los datasets según lo que generamos en la práctica 2:
datasets_map = {
    # Datos Originales
    "original": {"folder": "Original", "suffix": ""},
    "originalPCA95": {"folder": "Original", "suffix": "_pca95"},
    "originalPCA80": {"folder": "Original", "suffix": "_pca80"},
    
    # Datos Estandarizados
    "estandarizado": {"folder": "Estandarizado", "suffix": ""},
    "estandarizadoPCA95": {"folder": "Estandarizado", "suffix": "_pca95"},
    "estandarizadoPCA80": {"folder": "Estandarizado", "suffix": "_pca80"},
    
    # Datos Normalizados
    "normalizado": {"folder": "Normalizado", "suffix": ""},
    "normalizadoPCA95": {"folder": "Normalizado", "suffix": "_pca95"},
    "normalizadoPCA80": {"folder": "Normalizado", "suffix": "_pca80"},
}

# Carpeta donde guardaremos los modelos entrenados
models_output_folder = "Modelos"
if not os.path.exists(models_output_folder):
    os.makedirs(models_output_folder)

In [2]:
def train_and_save_model(fold, method, dataset_name):
 
    # La función entrena un modelo específico (method) para un pliegue (fold) y dataset (dataset_name) dados, y lo guarda en disco.
    
    # Identificamos el archivo de datos
    info = datasets_map[dataset_name]
    filename = f"training{fold}{info['suffix']}.csv"
    filepath = os.path.join(info['folder'], filename)

    if not os.path.exists(filepath):
        print(f"Error: No se encuentra {filepath}")
        return

    # Cargamos los datos asumiendoo que la última columna es la etiqueta (la clase que sea)
    data = pd.read_csv(filepath, header=None)
    X_train = data.iloc[:, :-1].values
    y_train = data.iloc[:, -1].values.ravel() # ravel() para asegurar que sea un vector 1D

    # Inicializamos el modelo
    if method not in modelos_config:
        print(f"Método {method} no reconocido.")
        return

    clf = modelos_config[method]
    
    # Entrenamos el modelo
    clf.fit(X_train, y_train)
    
    # Se guarda el modelo
    model_filename = f"model_{fold}_{dataset_name}_{method}.pkl"
    model_path = os.path.join(models_output_folder, model_filename)
    
    joblib.dump(clf, model_path)
    # Mencionamos en el import de arriba que usamos joblib para guardar los modelos entrenados

In [3]:
# Bucle principal de entrenamiento
print("Iniciando entrenamiento...")

folds = [1, 2, 3, 4, 5]
methods = ["knn", "svm", "nb", "rf"]
datasets = list(datasets_map.keys())

total_combinations = len(folds) * len(methods) * len(datasets)
counter = 0

for fold in folds:
    for ds_name in datasets:
        for method in methods:
            train_and_save_model(fold, method, ds_name)
            counter += 1
            
print(f"\n¡Entrenamiento finalizado! Se han generado {counter} modelos en la carpeta '{models_output_folder}'.")

Iniciando entrenamiento...

¡Entrenamiento finalizado! Se han generado 180 modelos en la carpeta 'Modelos'.
